# **DETECTOR DE OBJETOS CON YOLO V11**

In [ ]:
# INSTALACIÓN DE LIBRERÍAS NECESARIASA
#pip install ultralytics
#pip install roboflow

In [ ]:
# carga de utils
import sys
sys.path.append('../utils')
from utils import generar_lineas_por_manzana, procesar_jsons_en_carpeta, tabulate_jsons_from_folder, copiar_archivos_seleccionados

In [ ]:
# carga de librerías
from ultralytics import YOLO
import pandas as pd
import numpy as np
import shutil

In [ ]:
# Ruta a la carpeta del dataset
size = 256
nombre_modelo = 'segformer-200-512'
prepath = '../data/raw/data_set_'

path = f'{prepath}{size}/'
print(path)

In [ ]:
# Procesamiento de la data de los labels
procesar_jsons_en_carpeta(f'{path}metadata_y_{size}/','../data/interim_yolo/dataset_labels/')

In [ ]:
# Definición de train y test
jsons = tabulate_jsons_from_folder(f'{path}metadata_y_{size}/')#('../data/raw/data_set_256/metadata_y_256/')

l1 = jsons[jsons['num_polygons_in_window']>=1]['id']
l0 = jsons[jsons['num_polygons_in_window']==0]['id'].sample(n=int(np.round(len(l1)/5)), random_state=42)
l = pd.concat([l1,l0])

l_train = l.sample(frac=0.8, random_state=42)
l_test = l.drop(l_train.index)

In [ ]:
# Adecuación al directorio requerido por Yolo
#copiar_archivos_seleccionados('../data/raw/data_set_256/dataset_x_256/', '../data/interim_yolo/train/images/', [str(x)+'.png' for x in l_train])
#copiar_archivos_seleccionados('../data/raw/data_set_256/dataset_x_256/', '../data/interim_yolo/valid/images/', [str(x)+'.png' for x in l_test])

copiar_archivos_seleccionados(f'{path}dataset_x_{size}/', '../data/interim_yolo/train/images/', [str(x)+'.png' for x in l_train])
copiar_archivos_seleccionados(f'{path}dataset_x_{size}/', '../data/interim_yolo/valid/images/', [str(x)+'.png' for x in l_test])

copiar_archivos_seleccionados('../data/interim_yolo/dataset_labels/', '../data/interim_yolo/train/labels/', [str(x)+'.txt' for x in l_train])
copiar_archivos_seleccionados('../data/interim_yolo/dataset_labels/', '../data/interim_yolo/valid/labels/', [str(x)+'.txt' for x in l_test])

In [ ]:
# DESCARGA DE DATASET PERSONALIZADO DESDE ROBOFLOW
#!pip install roboflow

#from roboflow import Roboflow
#rf = Roboflow(api_key="YP6UX0p6bmRnwAjcSbfY")
#project = rf.workspace("nella-hyakz").project("custom_object_detector_yolo11_v2-rotnm")
#version = project.version(1)
#dataset = version.download("yolov11")
                

In [ ]:
# CARGA DEL MODELO BASE YOLO V11
from ultralytics import YOLO
model = YOLO("yolo11s.pt")

In [ ]:
# ENTRENAMIENTO DEL MODELO PERSONALIZADO
data_path = "custom_object_detector_yolo11_v2-1/data.yaml"
results= model.train (data=data_path,
epochs=15,
imgsz=size)

# HASTA ACÁ ESTÁ BIEN QUE CORRA, PUSHEA EL REPO Y YO ME ENCARGO

# Predicciones

In [ ]:
# CARGAR EL MODELO YA ENTRENADO
custom_model = YOLO("runs/detect/train/weights/best.pt")

In [ ]:
# REALIZAR PREDICCIONES SOBRE ALGUNAS IMÁGENES
res = custom_model("custom_object_detector_yolo11_v2-1/test/images")

In [ ]:
# VISUALIZAR LOS RESULTADOS DE LAS DETECCIONES
res[0].show()

In [ ]:
# VISUALIZAR LOS RESULTADOS DE LAS DETECCIONES
res[1].show()